In [ ]:
#task 1 and 2

In [ ]:
!pip install flask

import os
import sqlite3
import json
from flask import Flask, request, jsonify
from google.colab import output
import threading
import time

app = Flask(__name__)

def setup_database():
    """Create the SQLite database and table for storing app information."""
    conn = sqlite3.connect('app_database.db')
    cursor = conn.cursor()

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS apps (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        app_name TEXT NOT NULL,
        version TEXT NOT NULL,
        description TEXT
    )
    ''')

    sample_apps = [
        ('Calculator', '1.0', 'A simple calculator app'),
        ('Notes', '2.1', 'App for taking quick notes'),
        ('Weather', '3.5', 'Shows current weather information')
    ]

    cursor.executemany('''
    INSERT INTO apps (app_name, version, description)
    VALUES (?, ?, ?)
    ''', sample_apps)

    conn.commit()
    conn.close()
    print("Database setup complete with sample data!")

@app.route('/add-app', methods=['POST'])
def add_app():
    """Add a new app to the database."""
    data = request.get_json()

    if not all(key in data for key in ['app_name', 'version', 'description']):
        return jsonify({
            'status': 'error',
            'message': 'Missing required fields (app_name, version, description)'
        }), 400

    conn = sqlite3.connect('app_database.db')
    cursor = conn.cursor()

    try:
        cursor.execute('''
        INSERT INTO apps (app_name, version, description)
        VALUES (?, ?, ?)
        ''', (data['app_name'], data['version'], data['description']))

        app_id = cursor.lastrowid
        conn.commit()

        return jsonify({
            'status': 'success',
            'message': 'App added successfully',
            'app_id': app_id
        }), 201

    except Exception as e:
        return jsonify({
            'status': 'error',
            'message': f'Error adding app: {str(e)}'
        }), 500

    finally:
        conn.close()

@app.route('/get-app/<int:id>', methods=['GET'])
def get_app(id):
    """Retrieve app details by ID."""
    conn = sqlite3.connect('app_database.db')
    cursor = conn.cursor()

    try:
        cursor.execute('SELECT id, app_name, version, description FROM apps WHERE id = ?', (id,))
        app = cursor.fetchone()

        if app:
            return jsonify({
                'status': 'success',
                'data': {
                    'id': app[0],
                    'app_name': app[1],
                    'version': app[2],
                    'description': app[3]
                }
            }), 200
        else:
            return jsonify({
                'status': 'error',
                'message': f'App with ID {id} not found'
            }), 404

    except Exception as e:
        return jsonify({
            'status': 'error',
            'message': f'Error retrieving app: {str(e)}'
        }), 500

    finally:
        conn.close()

@app.route('/delete-app/<int:id>', methods=['DELETE'])
def delete_app(id):
    """Remove an app by ID."""
    conn = sqlite3.connect('app_database.db')
    cursor = conn.cursor()

    try:
        cursor.execute('SELECT id FROM apps WHERE id = ?', (id,))
        app = cursor.fetchone()

        if not app:
            return jsonify({
                'status': 'error',
                'message': f'App with ID {id} not found'
            }), 404

        cursor.execute('DELETE FROM apps WHERE id = ?', (id,))
        conn.commit()

        return jsonify({
            'status': 'success',
            'message': f'App with ID {id} deleted successfully'
        }), 200

    except Exception as e:
        return jsonify({
            'status': 'error',
            'message': f'Error deleting app: {str(e)}'
        }), 500

    finally:
        conn.close()

def run_flask_app():
    """Run the Flask app with Colab's port forwarding."""
    setup_database()

    app.run(port=5000)

def start_app():
    """Start the Flask app in a background thread."""
    flask_thread = threading.Thread(target=run_flask_app)
    flask_thread.daemon = True
    flask_thread.start()

    time.sleep(3)

    output.serve_kernel_port_as_window(5000)

    print("API server is running!")
    print("You can now access the API endpoints.")
    print("\nTest commands (run in a new cell):")
    print("""
# Add a new app
!curl -X POST http://localhost:5000/add-app -H "Content-Type: application/json" -d '{"app_name":"TestApp", "version":"1.0", "description":"Test application"}'

# Get an app (replace 1 with an actual ID)
!curl http://localhost:5000/get-app/1

# Delete an app (replace 1 with an actual ID)
!curl -X DELETE http://localhost:5000/delete-app/1
    """)

start_app()

Database setup complete with sample data!
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


Try `serve_kernel_port_as_iframe` instead. 


<IPython.core.display.Javascript object>

API server is running!
You can now access the API endpoints.

Test commands (run in a new cell):

# Add a new app
!curl -X POST http://localhost:5000/add-app -H "Content-Type: application/json" -d '{"app_name":"TestApp", "version":"1.0", "description":"Test application"}'

# Get an app (replace 1 with an actual ID)
!curl http://localhost:5000/get-app/1

# Delete an app (replace 1 with an actual ID)
!curl -X DELETE http://localhost:5000/delete-app/1
    


In [ ]:
!curl -X POST http://localhost:5000/add-app -H "Content-Type: application/json" -d '{"app_name":"TestApp", "version":"1.0", "description":"Test application"}'

!curl http://localhost:5000/get-app/1

!curl -X DELETE http://localhost:5000/delete-app/1

!curl http://localhost:5000/get-app/1

INFO:werkzeug:127.0.0.1 - - [16/Apr/2025 03:58:33] "POST /add-app HTTP/1.1" 201 -


{"app_id":4,"message":"App added successfully","status":"success"}


INFO:werkzeug:127.0.0.1 - - [16/Apr/2025 03:58:33] "GET /get-app/1 HTTP/1.1" 200 -


{"data":{"app_name":"Calculator","description":"A simple calculator app","id":1,"version":"1.0"},"status":"success"}


INFO:werkzeug:127.0.0.1 - - [16/Apr/2025 03:58:34] "DELETE /delete-app/1 HTTP/1.1" 200 -


{"message":"App with ID 1 deleted successfully","status":"success"}


INFO:werkzeug:127.0.0.1 - - [16/Apr/2025 03:58:34] "GET /get-app/1 HTTP/1.1" 404 -


{"message":"App with ID 1 not found","status":"error"}


In [ ]:
#task 3

In [ ]:
!pip install psutil

import psutil
import random
import os
import time

class VirtualAndroid:
    def __init__(self):
        """Initialize the virtual Android system with simulated properties."""
        self.os_version = "Android 13"
        self.device_model = "Pixel 6 (Simulated)"
        self.installed_apps = []
        self.total_memory = 8 * 1024
        self.used_memory = random.randint(1500, 3000)
        self.available_memory = self.total_memory - self.used_memory
        self.system_info = {
            "os_version": self.os_version,
            "device_model": self.device_model,
            "total_memory_mb": self.total_memory,
            "used_memory_mb": self.used_memory,
            "available_memory_mb": self.available_memory,
            "cpu_cores": psutil.cpu_count(),
            "cpu_usage": psutil.cpu_percent(),
        }

    def start(self):
        """Start the virtual Android system."""
        print(f"\n{'=' * 50}")
        print(f"Starting {self.os_version} on {self.device_model}")
        print(f"{'=' * 50}")
        time.sleep(1)
        print("System booting...")
        time.sleep(1)
        print("Loading system services...")
        time.sleep(1)
        print("Virtual Android System is now running!\n")
        self.display_system_info()

    def display_system_info(self):
        """Display system information in a formatted way."""
        print(f"\n{'=' * 30} SYSTEM INFO {'=' * 30}")
        print(f"OS Version: {self.os_version}")
        print(f"Device Model: {self.device_model}")
        print(f"CPU Cores: {self.system_info['cpu_cores']}")
        print(f"CPU Usage: {self.system_info['cpu_usage']}%")
        print(f"Memory: {self.used_memory} MB used / {self.total_memory} MB total")
        print(f"Available Memory: {self.available_memory} MB")
        print(f"Installed Apps: {len(self.installed_apps)}")
        print(f"{'=' * 70}\n")

        with open('android_system_log.txt', 'w') as f:
            for key, value in self.system_info.items():
                f.write(f"{key}: {value}\n")
            if self.installed_apps:
                f.write(f"installed_apps: {', '.join(self.installed_apps)}\n")

        print("System information has been logged to 'android_system_log.txt'")

    def install_app(self, apk_path):
        """
        Simulate installing an app from an APK file.

        Args:
            apk_path (str): Path to the APK file (simulated)
        """
        app_name = os.path.basename(apk_path).replace('.apk', '')

        print(f"\nInstalling {app_name}...")
        time.sleep(2)
        print("Verifying package...")
        time.sleep(1)
        print("Extracting files...")
        time.sleep(1)

        self.installed_apps.append(app_name)
        print(f"{app_name} installed successfully!")

        app_size = random.randint(50, 200)
        self.used_memory += app_size
        self.available_memory -= app_size
        self.system_info["used_memory_mb"] = self.used_memory
        self.system_info["available_memory_mb"] = self.available_memory

        print(f"App size: {app_size} MB")
        print(f"Updated available memory: {self.available_memory} MB")

        with open('android_system_log.txt', 'w') as f:
            for key, value in self.system_info.items():
                f.write(f"{key}: {value}\n")
            f.write(f"installed_apps: {', '.join(self.installed_apps)}\n")

with open('sample_app.apk', 'w') as f:
    f.write("This is a simulated APK file for demonstration purposes.")

android = VirtualAndroid()
android.start()

print("\nTo install an app, use the following command:")
print("android.install_app('sample_app.apk')")

android.install_app('sample_app.apk')


Starting Android 13 on Pixel 6 (Simulated)
System booting...
Loading system services...
Virtual Android System is now running!


============================== SYSTEM INFO ==============================
OS Version: Android 13
Device Model: Pixel 6 (Simulated)
CPU Cores: 2
CPU Usage: 67.2%
Memory: 2401 MB used / 8192 MB total
Available Memory: 5791 MB
Installed Apps: 0

System information has been logged to 'android_system_log.txt'

To install an app, use the following command:
android.install_app('sample_app.apk')

Installing sample_app...
Verifying package...
Extracting files...
sample_app installed successfully!
App size: 181 MB
Updated available memory: 5610 MB


In [ ]:
#task 4

In [ ]:
import requests
import json
import time
import random

def generate_device_id():
    """Generate a random device ID."""
    return f"ANDROID-{random.randint(10000, 99999)}"

def collect_system_info():
    """Collect simulated system information."""
    return {
        "device_id": generate_device_id(),
        "os_version": "Android 13",
        "device_model": "Pixel 6 (Simulated)",
        "total_memory_mb": 8 * 1024,
        "used_memory_mb": random.randint(1500, 3000),
        "available_memory_mb": random.randint(4000, 6000),
        "cpu_cores": 8,
        "cpu_usage": random.randint(5, 80),
        "battery_level": random.randint(20, 100),
        "network_type": random.choice(["WiFi", "4G", "5G"]),
        "timestamp": time.time()
    }

print("Starting networking script...")
print("Collecting system information...")

system_info = collect_system_info()
print(f"\nSystem information collected:")
for key, value in system_info.items():
    print(f"  {key}: {value}")

with open('network_log.txt', 'w') as log_file:
    log_file.write("=== NETWORKING OPERATIONS LOG ===\n\n")

    try:
        print("\nConnecting to backend server...")
        log_file.write("Connecting to backend server...\n")

        app_data = {
            "app_name": f"AndroidDevice-{system_info['device_id']}",
            "version": system_info['os_version'],
            "description": f"Data from {system_info['device_model']}"
        }

        print(f"Sending data to server at http://localhost:5000/add-app...")
        log_file.write(f"Sending data to server at http://localhost:5000/add-app...\n")
        log_file.write(f"Payload: {json.dumps(app_data)}\n")

        response = requests.post(
            "http://localhost:5000/add-app",
            json=app_data,
            headers={"Content-Type": "application/json"}
        )

        print(f"Server response status: {response.status_code}")
        log_file.write(f"Server response status: {response.status_code}\n")

        if response.status_code == 201:
            response_data = response.json()
            print(f"Server response: {json.dumps(response_data, indent=2)}")
            log_file.write(f"Server response: {json.dumps(response_data, indent=2)}\n")

            app_id = response_data['app_id']
            print(f"\nRetrieving data with ID {app_id}...")
            log_file.write(f"\nRetrieving data with ID {app_id}...\n")

            get_response = requests.get(f"http://localhost:5000/get-app/{app_id}")
            print(f"GET response status: {get_response.status_code}")
            log_file.write(f"GET response status: {get_response.status_code}\n")

            if get_response.status_code == 200:
                get_data = get_response.json()
                print(f"Retrieved data: {json.dumps(get_data, indent=2)}")
                log_file.write(f"Retrieved data: {json.dumps(get_data, indent=2)}\n")

    except requests.RequestException as e:
        print(f"\nNetwork error: {str(e)}")
        print("This is expected if the Flask server is not running.")
        log_file.write(f"\nNetwork error: {str(e)}\n")
        log_file.write("This is expected if the Flask server is not running.\n")

        log_file.write("\n=== SIMULATED SUCCESSFUL CONNECTION ===\n")
        log_file.write("Since the Flask server might not be running, here's what a successful connection would look like:\n\n")

        simulated_response = {
            "status": "success",
            "message": "App added successfully",
            "app_id": 4
        }
        log_file.write(f"Simulated response: {json.dumps(simulated_response, indent=2)}\n")

print("\nNetworking operations completed!")
print("Check 'network_log.txt' for detailed logs.")

INFO:werkzeug:127.0.0.1 - - [15/Apr/2025 18:04:19] "POST /add-app HTTP/1.1" 201 -
INFO:werkzeug:127.0.0.1 - - [15/Apr/2025 18:04:19] "GET /get-app/12 HTTP/1.1" 200 -


Starting networking script...

System information collected:
  device_id: ANDROID-95552
  os_version: Android 13
  device_model: Pixel 6 (Simulated)
  total_memory_mb: 8192
  used_memory_mb: 1782
  available_memory_mb: 4638
  cpu_cores: 8
  cpu_usage: 23
  battery_level: 24
  network_type: 5G
  timestamp: 1744740259.4856415

Connecting to backend server...
Sending data to server at http://localhost:5000/add-app...
Server response status: 201
Server response: {
  "app_id": 12,
  "message": "App added successfully",
  "status": "success"
}

Retrieving data with ID 12...
GET response status: 200
Retrieved data: {
  "data": {
    "app_name": "AndroidDevice-ANDROID-95552",
    "description": "Data from Pixel 6 (Simulated)",
    "id": 12,
    "version": "Android 13"
  },
  "status": "success"
}

Networking operations completed!
Check 'network_log.txt' for detailed logs.
